# Import Dependencies

In [162]:
# import libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [163]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/faisal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data

## Collection

In [164]:
# Collect data using kaggle api
! kaggle competitions download -c fake-news
! unzip fake-news.zip

100%|██████████████████████████████████████| 46.5M/46.5M [00:14<00:00, 3.46MB/s]
100%|██████████████████████████████████████| 46.5M/46.5M [00:14<00:00, 3.45MB/s]
Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [165]:
news_dataset_train = pd.read_csv("train.csv")
news_dataset_test = pd.read_csv("test.csv")

In [166]:
# shape of the dataset

news_dataset_train.shape

(20800, 5)

In [167]:
# viewing first few rows of the dataset

news_dataset_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [168]:
# checking for missing values

news_dataset_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [169]:
# replacing null values with empty string

news_dataset_train = news_dataset_train.fillna("")

In [170]:
# combine title and author
news_dataset_train['content'] = news_dataset_train['title'] + ' ' + news_dataset_train['author']

In [171]:
print(news_dataset_train['content'])

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799              What Keeps the F-35 Alive David Swanson
Name: content, Length: 20800, dtype: object


In [172]:
# Separating features and label

X = news_dataset_train.drop("label", axis = 1)
Y = news_dataset_train["label"]

## Stemming

In [173]:
port_stem = PorterStemmer()

In [174]:
# stemming function
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [175]:
# applying stemming function
news_dataset_train['content'] = news_dataset_train['content'].apply(stemming)

In [176]:
# viewing content after stemming
print(news_dataset_train['content'])

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: content, Length: 20800, dtype: object


In [177]:
# separating data and label

X = news_dataset_train['content'].values
Y = news_dataset_train['label'].values

In [178]:
# converting textual data to numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [179]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [180]:
# splitting data to training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

# Model

## Training

**Training model using Logistic Regression**

In [181]:
model = LogisticRegression()

In [182]:
model.fit(X_train, Y_train)

LogisticRegression()

## Evaluation

In [183]:
# accuracy score on training data
X_train_predictions = model.predict(X_train)
accuracy_training_data = accuracy_score(X_train_predictions, Y_train)
print(f"Accuracy score for training data: {accuracy_training_data}")

Accuracy score for training data: 0.9865985576923076


In [184]:
# accuracy score on test data
X_test_predictions = model.predict(X_test)
accuracy_test_data = accuracy_score(X_test_predictions, Y_test)
print(f"Accuracy score for test data: {accuracy_test_data}")

Accuracy score for test data: 0.9790865384615385


## Prediction for provided test data

In [185]:
news_dataset_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [186]:
news_dataset_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


In [187]:
news_dataset_test.shape

(5200, 4)

In [188]:
news_dataset_test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [189]:
news_dataset_test = news_dataset_test.fillna(" ")

In [190]:
# combine title and author
news_dataset_test['content'] = news_dataset_test['title'] + ' ' + news_dataset_test['author']

In [192]:
# applying stemming function
news_dataset_test['content'] = news_dataset_test['content'].apply(stemming)

In [198]:
# separating data

A = news_dataset_test['content'].values

In [200]:
# converting textual data to numerical data

A = vectorizer.transform(A)

A_predictions = model.predict(A)

print(A_predictions)

[0 1 1 ... 0 1 0]


In [202]:
#converting predictions to a dataframe
 
predictions_df = pd.DataFrame(A_predictions)
predictions_df.columns =['label']
predictions_df.head()

,label
0,0
1,1
2,1
3,0
4,1


In [203]:
df = pd.concat([news_dataset_test["id"], predictions_df], axis=1)
df.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [205]:
df.to_csv("answer.csv", index = False)